In [ ]:
import os
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('ggplot')

In [ ]:
train_df = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")

feature_df = pd.read_csv("../input/nbme-score-clinical-patient-notes/features.csv")
patient_df = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")

In [ ]:
print(train_df.shape)
print(feature_df.shape)
print(patient_df.shape)

lets check the datatypes for the above dataframes

In [ ]:
train_df.info()

In [ ]:
print( train_df.annotation.apply(type).eq(str).all() )
print( train_df.location.apply(type).eq(str).all() )

In [ ]:
train_df.annotation = train_df.annotation.apply(eval)
train_df.location   = train_df.location.apply(eval)

train_df.head()

In [ ]:
patient_df.head(2)

In [ ]:
print("Number of Patient Notes in the train dataset:", train_df.pn_num.nunique())
print("Number of Cases:", train_df.case_num.nunique())

print("number of patient notes colllected in total:", patient_df.pn_num.nunique())

In [ ]:
train_df['num_annotations'] = train_df.annotation.apply(len)
train_df.head(2)

In [ ]:
print("Number of samples with zero annotations:", len(train_df[train_df.num_annotations==0]))
print("percent of samples with zero annotations:{:.2f}".format(100 * len(train_df[train_df.num_annotations==0])/len(train_df) ))


In [ ]:
sns.countplot(data=train_df[train_df.num_annotations>0], x='num_annotations')
plt.show()

In [ ]:
for num_annotations in range(0, np.max(train_df.num_annotations)):
    c = len(train_df[train_df.num_annotations == num_annotations])
    p = c/len(train_df)
    print(num_annotations)
    print(c, p)    

In the train dataset around 87.9% of the samples will have either (0 or 1) annotations per (patient, case, feature)

In [ ]:
data = train_df.groupby('pn_num', as_index=False)[['case_num']].agg(['unique']).reset_index()
data.columns=['pn_num', 'case_num']
data['num_cases'] = data['case_num'].apply(len)

data.head(2)

In [ ]:
print( "Number of cases per patient notes:", data.num_cases.nunique() )

lets see the patient notes.

In [ ]:
print("number of patient notes:", len(patient_df))

In [ ]:
patient_df.isna().sum()

In [ ]:
patient_df.head()

In [ ]:
data = patient_df.groupby('case_num', as_index=False)[['pn_num']].count()

plt.title("distribution of patient notes for each case.")
sns.barplot(data=data, x='case_num', y='pn_num')
plt.show()

In [ ]:
patient_df['num_words'] = patient_df.pn_history.apply(lambda s: len(s.split()))
patient_df.head()

In [ ]:
patient_df.num_words.describe()

In [ ]:
plt.title("distribution on the number of words per notes")
sns.histplot(data=patient_df, x='num_words')
plt.show()

# features data

In [ ]:
data = feature_df.groupby('case_num', as_index=False)[['feature_num']].count()

plt.title("distribution of features per case.")
sns.barplot(data=data, x='case_num', y='feature_num')
plt.show()

In [ ]:
train_df.columns

In [ ]:
case_annotation_df=[]

for _, row in train_df.iterrows():
    case_num = row.case_num
    annotations = row.annotation
    
    for annot in annotations:
        case_annotation_df.append({
            'case_num': case_num,
            'annot_len': len(annot.split())
        })
case_annotation_df = pd.DataFrame.from_dict(case_annotation_df)
case_annotation_df.head()

In [ ]:
case_annotation_df.annot_len.describe()

In [ ]:
sns.histplot(data=case_annotation_df, x='annot_len')

# Notes 

In [ ]:
print(patient_df.pn_history.sample().values[0])

Few things that might to try out.
1. Age of the person.
2. Abbrevations such as PMH, PSH etc.
3. Time(since when experiencing the condition)